In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import numpy as np
from minitorch.tensor.tensor import Tensor
from minitorch.nn.layers import (Linear,
                                Dropout,
                                Sequential,
                                Residual,
                                BatchNormalization,
                                LayerNormalization)

In [3]:
batch, in_features, out_features = 5,6,5
layer1 = Linear(in_features, out_features, bias=True)
layer2 = Linear(out_features, in_features, bias=True)
layer3 = Linear(in_features, 4, bias=True)

drop_out = Dropout(0.2, training=True)

d_model = Sequential([layer1,drop_out, layer2,drop_out, layer3])
nd_model = Sequential([layer1, layer2, layer3])

x = Tensor(np.random.randn(batch, in_features))
print(x)
d_out = d_model(x)
nd_out = nd_model(x)

Tensor(data=[[ 0.60861002  0.02777523 -2.50207599  0.02443978  0.15802738 -0.32984906]
 [ 0.62397222 -0.43682316  0.89401898 -0.5977046  -0.47511292  0.30087595]
 [-0.67911549 -2.15712305 -0.22901759  0.87133232  0.51409134  1.43558754]
 [ 1.23051082 -0.5773788  -0.77971186  0.69407013 -0.02735502 -1.03969185]
 [-0.27782297  1.04613403  0.36037149 -1.43293299  1.42400674 -0.93866863]])


In [4]:
weight, bias = layer1.parameters()

In [5]:
layer1.__repr__()

'Linear(in_features=6, out_features=5, bias=True)'

In [6]:
bn = BatchNormalization(num_features=d_out.shape[-1])
out_norm = bn(d_out)

In [7]:
lm = LayerNormalization(num_features=d_out.shape[-1])
out_lnorm = lm(d_out)

In [8]:
out_norm, out_lnorm

(Tensor(data=[[-0.42215618 -1.14602737 -1.23274619 -0.4452985 ]
  [ 0.38421401 -0.16490078 -0.55692575 -0.69299184]
  [-0.34259789 -0.24803059  0.46104468 -0.92649303]
  [-1.12959271 -0.05200027 -0.06879217  0.62544728]
  [ 1.51013276  1.610959    1.39741944  1.43933608]], shape=(5, 4), grad_info= requires_grad=True),
 Tensor(data=[[ 0.89379028 -1.32884732 -0.19516003  0.63021707]
  [ 1.47424078 -0.3969503  -0.74901706 -0.32827342]
  [ 0.60876542 -0.92207159  1.08666303 -0.77335686]
  [-0.74682144 -0.41315698 -0.31393335  1.47391177]
  [-0.50151271  1.493448   -0.38175685 -0.61017844]], shape=(5, 4), grad_info= requires_grad=True))

In [9]:
TOLERANCE = 1e-3

def unit_test():
    print("="*50)
    print("🧪Running unit tests for Linear layer...")
    in_features, out_features = 784, 256
    layer = Linear(in_features, out_features, bias=True)
    
    #* test layer creation
    assert layer.weight.shape == (out_features, in_features), "Weight shape is incorrect."
    assert layer.bias.shape == (out_features,), "Bias shape is incorrect."
    assert layer.weight.requires_grad, "Weight should require gradients."
    assert layer.bias.requires_grad, "Bias should require gradients."
    print("🧪 Layer creation tests passed ...")
    
    #* test Xavier initialization
    weight_std = np.std(layer.weight.data)
    expected_std = (1.0 / in_features) ** 0.5
    assert np.isclose(weight_std, expected_std, atol=TOLERANCE), "Weight initialization is not Xavier."
    print("🧪 Xavier initialization tests passed ...")
    
    #* bias tests
    assert np.all(layer.bias.data == 0), "Bias should be initialized to zeros."
    layer_no_bias = Linear(in_features, out_features, bias=False)
    assert layer_no_bias.bias is None, "Bias should be None when bias=False."
    params = layer_no_bias.parameters()
    assert len(params) == 1 and params[0] is layer_no_bias.weight, "Parameters should only include weight when bias=False."
    print("🧪 Bias initialization tests passed ...")
    
    #* forward pass tests
    batch_size = 10
    x = Tensor(np.random.randn(batch_size, in_features))
    output = layer(x)
    assert output.shape == (batch_size, out_features), f"expected output shape {(batch_size, out_features)}, got {output.shape}."
    print("🧪 Forward pass tests passed ...")
    print("✅ All unit tests passed!")
    print("="*50)
    
unit_test()

🧪Running unit tests for Linear layer...
🧪 Layer creation tests passed ...
🧪 Xavier initialization tests passed ...
🧪 Bias initialization tests passed ...
🧪 Forward pass tests passed ...
✅ All unit tests passed!


In [10]:
def test_edge_cases_linear():
    """🔬 Test Linear layer edge cases."""
    print("🔬 Edge Case Tests: Linear Layer...")

    layer = Linear(10, 5)

    # Test single sample (should handle 2D input)
    x_2d = Tensor(np.random.randn(1, 10))
    y = layer.forward(x_2d)
    assert y.shape == (1, 5), "Should handle single sample"
    print("🧪 Single sample handled correctly...")
    
    # Test zero batch size (edge case)
    x_empty = Tensor(np.random.randn(0, 10))
    y_empty = layer.forward(x_empty)
    assert y_empty.shape == (0, 5), "Should handle empty batch"
    print("🧪 Empty batch handled correctly...")
    
    # Test numerical stability with large weights
    layer_large = Linear(10, 5)
    layer_large.weight.data = np.ones((5,10)) * 100  # Large but not extreme
    x = Tensor(np.ones((1, 10)))
    y = layer_large.forward(x)
    assert not np.any(np.isnan(y.data)), "Should not produce NaN with large weights"
    assert not np.any(np.isinf(y.data)), "Should not produce Inf with large weights"
    print("🧪 Numerical stability with large weights handled correctly...")
    
    # Test with no bias
    layer_no_bias = Linear(10, 5, bias=False)
    x = Tensor(np.random.randn(4, 10))
    y = layer_no_bias.forward(x)
    assert y.shape == (4, 5), "Should work without bias"
    print("🧪 No bias case handled correctly...")
    print("✅ Edge cases handled correctly!")
    print("="*50)
    
test_edge_cases_linear()

🔬 Edge Case Tests: Linear Layer...
🧪 Single sample handled correctly...
🧪 Empty batch handled correctly...
🧪 Numerical stability with large weights handled correctly...
🧪 No bias case handled correctly...
✅ Edge cases handled correctly!


In [11]:
def test_unit_dropout():
    """🧪 Unit tests for Dropout layer."""
    print("="*50)
    print("🧪Running unit tests for Dropout layer...")
    
    #* test layer creation
    p = 0.3
    layer = Dropout(p)
    assert layer.p == p, "Dropout probability not set correctly."
    print("🧪 Layer creation tests passed ...")
    
    #* test forward pass in training mode
    batch_size, features = 10, 5
    x = Tensor(np.ones((batch_size, features)))
    output = layer(x)
    dropped_elements = np.sum(output.data == 0)
    expected_dropped = int(batch_size * features * p)
    assert abs(dropped_elements - expected_dropped) < batch_size * features * 0.1, "Dropout did not drop the expected number of elements."
    print("🧪 Forward pass in training mode tests passed ...")
    
    #* test forward pass in evaluation mode
    output_eval = layer(x, training=False)
    assert np.all(output_eval.data == x.data), "Dropout should not modify input in evaluation mode."
    print("🧪 Forward pass in evaluation mode tests passed ...")
    
    #* test training mode with p=0 (no dropout)
    layer_no_dropout = Dropout(0.0)
    output_no_dropout = layer_no_dropout(x)
    assert np.all(output_no_dropout.data == x.data), "Dropout with p=0 should not modify input."
    print("🧪 No dropout (p=0) tests passed ...")
    
    #* test training mode with p=1 (all dropped)
    layer_all_dropout = Dropout(1.0 - 1e-6) # avoid exact 1.0 to prevent division by zero
    output_all_dropout = layer_all_dropout(x)
    assert np.all(output_all_dropout.data == 0), "Dropout with p=1 should drop all elements."
    print("🧪 All dropout (p=1) tests passed ...")
    
    #* test training mode with partial dropout
    #* this probabilistic test may fail occasionally due to randomness
    #* so we test statistical properties instead of exact values
    np.random.seed(42) # for reproducibility
    x_large = Tensor(np.ones((1000,))) # large input for statistical test
    layer_partial_dropout = Dropout(0.5)
    y_train = layer_partial_dropout(x_large, training=True)
    
    #* count non-dropped elements(approx 50% should remain)
    non_zero_count = np.count_nonzero(y_train.data)
    expected = 500
    #* Use 3-sigma bounds: std = sqrt(n*p*(1-p)) = sqrt(1000*0.5*0.5) ≈ 15.8
    std_error = np.sqrt(1000 * 0.5 * 0.5)
    lower_bound = expected - 3 * std_error  # ≈ 453
    upper_bound = expected + 3 * std_error  # ≈ 547
    assert lower_bound < non_zero_count < upper_bound, \
        f"Expected {expected}±{3*std_error:.0f} survivors, got {non_zero_count}"
    print("🧪 Partial dropout tests passed ...")
    
    # Test scaling (surviving elements should be scaled by 1/(1-p) = 2.0)
    surviving_values = y_train.data[y_train.data != 0]
    expected_value = 2.0  # 1.0 / (1 - 0.5)
    assert np.allclose(surviving_values, expected_value), f"Surviving values should be {expected_value}"
    print("🧪 Dropout scaling tests passed ...")
    
    print("🧪 Dropout layer tests passed ...")
    print("✅ All unit tests passed!")
    print("="*50)
    
test_unit_dropout()

🧪Running unit tests for Dropout layer...
🧪 Layer creation tests passed ...


AssertionError: Dropout did not drop the expected number of elements.

In [ ]:
import numpy as np
import pytest


# ----------------------
# Linear Layer Tests
# ----------------------

def test_linear_forward_shape():
    layer = Linear(4, 3)
    x = Tensor(np.random.randn(5, 4))

    out = layer(x)

    assert out.shape == (5, 3)
    print('Linear forward pass passed successfully')


def test_linear_backward_pass():
    layer = Linear(4, 2, bias=True)
    x = Tensor(np.random.randn(6, 4), requires_grad=True)

    out = layer(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None
    assert layer.weight.grad is not None
    assert layer.bias.grad is not None
    print('Linear backward pass passed successfully')
    


# ----------------------
# Dropout Tests
# ----------------------

def test_dropout_training_changes_values():
    dropout = Dropout(p=0.5)
    dropout.train()

    x = Tensor(np.ones((3, 10)))
    out = dropout(x)

    # Expect some zeros during training
    assert (out.data == 0).any(), 'There are no zeros in the tensor'
    print('Dropout training values test passed')


def test_dropout_eval_is_identity():
    dropout = Dropout(p=0.5)
    dropout.eval()

    x = Tensor(np.random.randn(10, 5))
    out = dropout(x)

    np.testing.assert_allclose(out.data, x.data)
    print('Drpout evaluation identity passed succesfuly')


# ----------------------
# Sequential Tests
# ----------------------

def test_sequential_forward_chain():
    model = Sequential([
        Linear(4, 6),
        Linear(6, 2),]
    )

    x = Tensor(np.random.randn(8, 4))
    out = model(x)

    assert out.shape == (8, 2)
    print('Sequential forward pass passed successfully')

def test_sequential_backward_pass():
    model = Sequential(
        [Linear(3, 5),
        Linear(5, 1),]
    )

    x = Tensor(np.random.randn(7, 3), requires_grad=True)
    out = model(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None
    print('Sequential backward pass passed')


# ----------------------
# Batch Normalization Tests
# ----------------------

def test_batchnorm_forward_shape():
    bn = BatchNormalization(num_features=4)
    x = Tensor(np.random.randn(10, 4))

    out = bn(x)

    assert out.shape == x.shape
    print('Batch normalization test passed successfully')


def test_batchnorm_running_stats_update():
    bn = BatchNormalization(num_features=3)
    bn.train()

    x = Tensor(np.random.randn(20, 3))

    running_mean_before = bn.running_mean.data.copy()
    _ = bn(x)

    assert not np.allclose(running_mean_before, bn.running_mean.data)
    print('Batch Normalization running stats update succesfull')


def test_batchnorm_eval_uses_running_stats():
    bn = BatchNormalization(num_features=2)

    x = Tensor(np.random.randn(10, 2))
    _ = bn(x)  # update stats

    bn.eval()
    out1 = bn(x)
    out2 = bn(x)

    np.testing.assert_allclose(out1.data, out2.data) # pyright: ignore[reportCallIssue]
    print('Batch Normalization evaluation uses passed')


# ----------------------
# Layer Normalization Tests
# ----------------------

def test_layernorm_forward_shape():
    ln = LayerNormalization(4)
    x = Tensor(np.random.randn(5, 4))

    out = ln(x)

    assert out.shape == x.shape
    print('Layer normalization forward pass passed')


def test_layernorm_zero_mean_unit_var():
    ln = LayerNormalization(6)
    x = Tensor(np.random.randn(12, 6))

    out = ln(x)

    mean = out.mean(axis=-1)
    var = out.var(axis=-1)

    np.testing.assert_allclose(actual=mean.data, desired=np.zeros_like(mean))
    np.testing.assert_allclose(actual=var.data, desired=np.ones_like(var))
    print('Layer normalization with zero mean and unit variance test passed')

def test_layernorm_backward_pass():
    ln = LayerNormalization(3)
    x = Tensor(np.random.randn(4, 3), requires_grad=True)

    out = ln(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None
    assert ln.weight.grad is not None
    assert ln.bias.grad is not None
    print('LayerNormalization backward test passed')

test_linear_forward_shape()
test_linear_backward_pass()
# test_dropout_training_changes_values()
test_dropout_eval_is_identity()
test_sequential_forward_chain()
test_sequential_backward_pass()
test_batchnorm_forward_shape()
test_batchnorm_eval_uses_running_stats()
# test_layernorm_zero_mean_unit_var()
test_layernorm_backward_pass()

Linear forward pass passed successfully
Linear backward pass passed successfully
Drpout evaluation identity passed succesfuly
Sequential forward pass passed successfully
Sequential backward pass passed
Batch normalization test passed successfully
Batch Normalization evaluation uses passed
Tensor(data=[ 3.51570624e-16  5.55111512e-17  5.55111512e-16  0.00000000e+00
  0.00000000e+00  1.29526020e-16  0.00000000e+00  0.00000000e+00
  4.62592927e-17 -8.32667268e-17 -4.62592927e-18 -3.70074342e-17]) Tensor(data=[0.99988861 0.99998825 0.99990978 0.99998483 0.999983   0.99999048
 0.99999197 0.99999398 0.99998196 0.99999601 0.99999675 0.99999025])


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
bias = Tensor(np.array([1., 1., 1.,]))
weight = Tensor(np.array([[-0.23160526,  1.09547947, -0.86387422],
 [-1.10820745 , 0.27320463,  0.83500282],
 [ 1.01400469, -0.98537308 ,-0.02863161],
 [-1.08807273  ,0.87880414,  0.20926859]]))

In [ ]:
weight + bias

Tensor(data=[[ 0.76839474  2.09547947  0.13612578]
 [-0.10820745  1.27320463  1.83500282]
 [ 2.01400469  0.01462692  0.97136839]
 [-0.08807273  1.87880414  1.20926859]], shape=(4, 3), grad_info= None)